In [1]:
import pandas as pd
import duckdb
import altair as alt
import numpy as np

In [2]:
# ¿Por que razones el voto no sumo a ningun partido?

elecciones_no_positivos = duckdb.sql("""
    SELECT eleccion_tipo, votos_tipo, SUM(votos_cantidad) AS votos 
    FROM read_parquet('ResultadosElectorales_*.parquet') 
    WHERE cargo_id = 1 AND votos_tipo != 'POSITIVO'
    GROUP BY 1, 2
""").df()

custom_palette = ['#6994A8','#C7C8C8','#215974']

elecciones_no_positivos['eleccion_tipo'] = pd.Categorical(elecciones_no_positivos['eleccion_tipo'], categories=['PASO', 'GENERAL', 'SEGUNDA VUELTA'], ordered=True)

# Ordena el DataFrame por 'eleccion_tipo' y 'votos_tipo'
elecciones_no_positivos = elecciones_no_positivos.sort_values(by=['eleccion_tipo', 'votos_tipo'])

chart = alt.Chart(elecciones_no_positivos).mark_bar().encode(
    y=alt.Y('votos_tipo:N', title=None, sort='-x'),  # Ordenar de forma descendente
    x=alt.X('sum(votos):Q', title='Cantidad de Votos'),
    color=alt.Color('eleccion_tipo:N', scale=alt.Scale(range=custom_palette)),
    order=alt.Order('votos_tipo:N', sort='descending')  # Ordenar internamente por 'votos_tipo'
).properties(
    width=700,  # Ajustar el ancho del gráfico según tus preferencias
    height=300
).configure_legend(
    orient='top',  # Posicionar la leyenda en la parte superior
    labelLimit=0  # Mostrar todas las etiquetas en la leyenda
)

chart

alt.Chart(...)

In [3]:
# Comparando las 3 elecciones, ¿cuantos votos recibieron  los partidos más relevantes?

def create_chart(df, title):
    df['votos_formateados'] = df['votos'].apply(lambda x: '{:,}'.format(x))
    base = alt.Chart(df).encode(
        x=alt.X('votos:Q', title="Votos"),
        y=alt.Y('agrupacion_nombre:N', sort='-x', title=None),
        color=alt.Color('agrupacion_nombre:N', scale=alt.Scale(domain=list(colors_dict.keys()), range=list(colors_dict.values())), legend=None),
    )
    bars = base.mark_bar().properties(
        title=f"Presidente, {title} 2023"
    )
    text = base.mark_text(align='left', dx=2, color='black').encode(
        text=alt.Text('votos_formateados:N')
    )
    return bars + text

colors_dict = {
    "JUNTOS POR EL CAMBIO": "#fedd00",
    "UNION POR LA PATRIA": "#009cde",
    "LA LIBERTAD AVANZA": "#753bbc",
    "FRENTE DE IZQUIERDA": "#f95461",
    "HACEMOS POR NUESTRO PAIS": "#43488f"
}

df_PASO = duckdb.sql("""
    SELECT agrupacion_nombre, sum(votos_cantidad) as votos
    FROM read_parquet('ResultadosElectorales_PASO_2023.parquet') 
    WHERE agrupacion_nombre IS NOT NULL AND cargo_id = 1
    GROUP BY agrupacion_nombre
    ORDER BY 2 desc
    limit 5
""").df()

df_GENERAL = duckdb.sql("""
    SELECT agrupacion_nombre, sum(votos_cantidad) as votos
    FROM read_parquet('ResultadosElectorales_1v.parquet') 
    WHERE agrupacion_nombre IS NOT NULL AND cargo_id = 1
    GROUP BY agrupacion_nombre
    ORDER BY 2 desc
""").df()

df_SEGUNDA_VUELTA = duckdb.sql("""
    SELECT agrupacion_nombre, sum(votos_cantidad) as votos
    FROM read_parquet('ResultadosElectorales_2023_SegundaVuelta.parquet') 
    WHERE agrupacion_nombre IS NOT NULL AND cargo_id = 1
    GROUP BY agrupacion_nombre
    ORDER BY 2 desc
""").df()

df_PASO['agrupacion_nombre'] = df_PASO['agrupacion_nombre'].replace(
    {'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD': 'FRENTE DE IZQUIERDA'})
df_GENERAL['agrupacion_nombre'] = df_GENERAL['agrupacion_nombre'].replace(
    {'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD': 'FRENTE DE IZQUIERDA'})
df_SEGUNDA_VUELTA['agrupacion_nombre'] = df_SEGUNDA_VUELTA['agrupacion_nombre'].replace(
    {'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD': 'FRENTE DE IZQUIERDA'})

chart_PASO = create_chart(df_PASO, "PASO")
chart_GENERAL = create_chart(df_GENERAL, "GENERAL")
chart_SEGUNDA_VUELTA = create_chart(df_SEGUNDA_VUELTA, "SEGUNDA VUELTA")

final_chart = alt.concat(chart_PASO, chart_GENERAL, chart_SEGUNDA_VUELTA, spacing=5)
final_chart.configure_axis(labelFontSize=12, titleFontSize=14).configure_title(fontSize=16).resolve_scale(color='independent').configure_view(continuousHeight=1500)

alt.ConcatChart(...)

In [4]:
# Comparando las 3 elecciones, ¿cuantos votos recibieron  los partidos más relevantes?

datos = duckdb.sql("""
    SELECT eleccion_tipo, agrupacion_nombre, sum(votos_cantidad) as votos
    FROM read_parquet('ResultadosElectorales_*.parquet') 
    WHERE agrupacion_nombre IN ('LA LIBERTAD AVANZA', 'JUNTOS POR EL CAMBIO','UNION POR LA PATRIA','HACEMOS POR NUESTRO PAIS','FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD')  
      AND agrupacion_nombre IS NOT NULL AND cargo_id = 1
    GROUP BY eleccion_tipo, agrupacion_nombre
    ORDER BY 
    votos DESC
""").df()

datos['agrupacion_nombre'] = datos['agrupacion_nombre'].replace({'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD': 'FRENTE DE IZQUIERDA'})

colors_dict = {
    "JUNTOS POR EL CAMBIO": "#fedd00",
    "UNION POR LA PATRIA": "#009cde",
    "LA LIBERTAD AVANZA": "#753bbc",
    "FRENTE DE IZQUIERDA": "#f95461",
    "HACEMOS POR NUESTRO PAIS": "#43488f"
}

orden_personalizado = ['PASO', 'GENERAL', 'SEGUNDA VUELTA']

# Crea el gráfico de líneas utilizando Altair con colores personalizados y orden personalizado en el eje X
chart = alt.Chart(datos).mark_line().encode(
    x=alt.X('eleccion_tipo:N', title=None, sort=orden_personalizado),  # 'N' indica que la variable es nominal (categórica sin orden)
    y=alt.Y('votos:Q', title=None),  # 'Q' indica que la variable es cuantitativa (numérica)
    color=alt.Color('agrupacion_nombre:N', scale=alt.Scale(domain=list(colors_dict.keys()), range=list(colors_dict.values()))),  # Asigna colores según el diccionario
).properties(width=600)  # Ajusta el ancho del gráfico según tus necesidades

# Configura el eje X e Y para que no tenga título y rote las etiquetas del eje X
chart = chart.configure_axis(
    labelFontSize=12,
    titleFontSize=12,
    labelAngle=0  # Ajusta el ángulo de las etiquetas del eje X
)

chart

alt.Chart(...)

In [5]:
# ¿Cuanta gente voto desde el extranjero en cada elección?

cant_extranjeros = duckdb.sql("""
    SELECT eleccion_tipo, SUM(votos_cantidad) AS votos 
    FROM read_parquet('ResultadosElectorales_*.parquet') 
    WHERE mesa_tipo = 'EXTRANJEROS' 
    GROUP BY eleccion_tipo
""").df()

colores_personalizados = ['#6994A8','#C7C8C8']

orden_categorias = ['PASO', 'GENERAL']

# Crea un gráfico de barras con colores personalizados y orden específico para cada barra
chart = alt.Chart(cant_extranjeros).mark_bar().encode(
    x=alt.X('eleccion_tipo:N', sort=orden_categorias),
    y=alt.Y('sum(votos):Q'),
    color=alt.Color('eleccion_tipo:N', scale=alt.Scale(range=colores_personalizados), legend=None)
).properties(width=600)

# Elimina el nombre de los ejes y ajusta la orientación de las etiquetas en el eje x
chart = chart.configure_axis(
    labelFontSize=12,  # Ajusta el tamaño de las etiquetas si es necesario
    title=None,        # Elimina el nombre de los ejes
    labelAngle=0,      # Establece el ángulo de las etiquetas en el eje x
)

chart

alt.Chart(...)

In [3]:
# ¿A quienes votaron las personas que están en el extranjero?

cant_extranjeros = duckdb.sql("""
    SELECT agrupacion_nombre, eleccion_tipo, SUM(votos_cantidad) AS votos, mesa_tipo
    FROM read_parquet('ResultadosElectorales_*.parquet') 
    WHERE agrupacion_nombre IN ('LA LIBERTAD AVANZA', 'JUNTOS POR EL CAMBIO', 'UNION POR LA PATRIA', 'HACEMOS POR NUESTRO PAIS', 'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD')  
    AND agrupacion_nombre IS NOT NULL  AND mesa_tipo = 'EXTRANJEROS'
    GROUP BY eleccion_tipo, agrupacion_nombre, mesa_tipo
""").df()

colores = {
    'PASO': '#C7C8C8',
    'GENERAL': '#6994A8'
}

chart = alt.Chart(cant_extranjeros).mark_bar().encode(
    y=alt.Y('agrupacion_nombre:N', title=None, sort='-x'), 
    x=alt.X('votos:Q', title=None),  
    color=alt.Color('eleccion_tipo:N', scale=alt.Scale(domain=list(colores.keys()), range=list(colores.values()))),  
    order=alt.Order('eleccion_tipo:N', sort='descending'), 
    tooltip=['agrupacion_nombre', 'votos', 'eleccion_tipo']
).properties(
    width=500, height=300
)

chart = chart.configure_axis(
    grid=False,  
    domain=False,  #
)
chart

alt.Chart(...)

In [4]:
# ¿Cual es la variación de votos entre las PASO y las GENERALES?

data = duckdb.sql("""
    SELECT agrupacion_nombre, sum(votos_cantidad) as votos, eleccion_tipo
    FROM read_parquet('ResultadosElectorales_*.parquet')
    where cargo_id=1
    GROUP BY agrupacion_nombre,eleccion_tipo
""").df()

data['agrupacion_nombre'] = data['agrupacion_nombre'].replace(
    {'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD': 'FRENTE DE IZQUIERDA'})


result_paso = data[data['eleccion_tipo'] == 'PASO'].groupby('agrupacion_nombre')['votos'].sum().reset_index()
result_paso.rename(columns={'votos': 'votos_paso'}, inplace=True)

result_generales = data[data['eleccion_tipo'] == 'GENERAL'].groupby('agrupacion_nombre')['votos'].sum().reset_index()
result_generales.rename(columns={'votos': 'votos_generales'}, inplace=True)

colores_partidos = {
    "JUNTOS POR EL CAMBIO": "#fedd00",
    "UNION POR LA PATRIA": "#009cde",
    "LA LIBERTAD AVANZA": "#753bbc",
    "FRENTE DE IZQUIERDA": "#f95461",
    "HACEMOS POR NUESTRO PAIS": "#43488f"
}

result_combined = pd.merge(result_paso, result_generales, on='agrupacion_nombre', how='inner')

result_combined['variacion_porcentaje'] = ((result_combined['votos_generales'] - result_combined['votos_paso']) / result_combined['votos_paso']) * 100


top_5_partidos = result_combined.nlargest(5, ['votos_paso', 'votos_generales'])
top_5_partidos = top_5_partidos.sort_values(by='variacion_porcentaje', ascending=False)

chart = alt.Chart(
    top_5_partidos,  # Usar el DataFrame directamente como origen de datos
    title='Variación de Votos - PASO y Generales'
).mark_bar().encode(
    y=alt.Y(
        'agrupacion_nombre:N',
        title='Partido Político',
        sort=alt.EncodingSortField(field='variacion_porcentaje', op='sum', order='descending')
    ),
    x=alt.X('variacion_porcentaje:Q', title='Variación de Votos (%)', scale=alt.Scale(domain=[-100, 100])),
    color=alt.Color('agrupacion_nombre:N', scale=alt.Scale(domain=list(colores_partidos.keys()), range=list(colores_partidos.values()))),
    tooltip=['agrupacion_nombre', 'variacion_porcentaje']
).properties(height=200)

final_chart = chart.interactive()
final_chart

alt.Chart(...)